In [5]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [6]:
import random

def bs_price(S,K,sigma,r,T, year_rate = 252, option_type = "call"):
    '''
    bs 期权定价 输出为期权价格
    输入格式：
    S:标的价格
    K:行权价
    sigma:波动率
    r:无风险利率
    T:年华剩余期限
    year_rate:年化系数
    '''

    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        # return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return np.maximum(S - K, 0) if T==0 else S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else :
        # return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        return np.maximum(K-S,0) if T==0 else K*np.exp(-r*T)*norm.cdf(-1*d2) - S*norm.cdf(-1*d1)

def bs_vega(S,K,sigma,r,T ,year_rate = 252):
    '''
    计算vega
    '''
    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(np.abs(T)) )
    return S*norm.pdf(d1)*np.sqrt(np.abs(T))
    # return S * np.sqrt(abs(T)) * np.exp(-pow(d1,2)/2) / np.sqrt(2*np.pi)

def iv_bs_bisection(S, K, r, T, price, option_type, iv_uplimit = 0.985, iv_downlimit = 0.015, precision = 3, year_rate = 365, max_iterations = 400):
    '''二分法求iv'''
    left_iv, right_iv = iv_downlimit, iv_uplimit
    # left_price = BS_price(S,K,left_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # right_price= BS_price(S,K,right_iv, r, T,  year_rate = year_rate, option_type = option_type)
    mid_iv = (left_iv + right_iv)/2
    mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # print(f"first, the mid price is {mid_price}, the price is {price}")
    cnt = 0
    while abs(price - mid_price) >= 0.1**precision and cnt < max_iterations:
        if mid_price < price :
            left_iv = mid_iv
        else:
            right_iv = mid_iv
        mid_iv = (left_iv + right_iv)/2
        # if mid_iv < iv_uplimit or mid_iv > iv_downlimit:
        #     break
        # print(f"cnt = {cnt}, the left is{left_iv}, the right is{right_iv}, the mid is {mid_iv}")
        cnt += 1
        mid_price = bs_price(S, K, mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    return mid_iv

def find_vol_newton( S, K, r, T, target_value, option_type, start_sigma = 0.5, precision = 3, year_rate = 252, max_iterations = 200):
    '''迭代法求iv'''
    sigma = start_sigma
    for i in range(0, max_iterations):
        bs_price_ = bs_price(S,K,sigma,r,T, year_rate = year_rate, option_type = option_type)
        vega = bs_vega(S, K, T, r, sigma, year_rate = year_rate)*100
        diff = target_value - bs_price_  # our root
        if (abs(diff) < 0.1**precision):
            return sigma
        sigma = sigma + diff/(vega) # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

def change(type_name):
    if type_name == '认购':
        return "call"
    elif type_name == '认沽':
        return "put"
    else:
        return "wrong_type"

In [8]:
#数据处理
alldata = []
rate = pd.read_excel(r".\data\unrisked_rate.xlsx")
rate['date'] = pd.to_datetime(rate['date'])
rate['Value'] = rate['Value']/100

etf50 = pd.read_excel(r'.\data\50etf_df.xlsx')
etf50 = etf50[['Date','close']]
etf50['Date'] = pd.to_datetime(etf50['Date'])

csv_folder = r'.\data\50etf_option_data_csv'
output_folder = r'.\data\out'
csv_list = os.listdir(csv_folder)
#picdata = []

for csv_name in csv_list:
    print(csv_name)
    perdata = []
    csv_path = csv_folder +"\\"+csv_name
    output_path = output_folder + "\\" + csv_name
    data = pd.read_csv(csv_path, encoding = 'GBK', index_col= False)

    data = data[['date','ptmtradeday','exe_price', 'open', 'high','low','close', 'call_or_put']]
    data["date"] = pd.to_datetime(data["date"])
    data['call_or_put'] = data['call_or_put'].apply(change)
    if data['open'].dtype == 'object':
        data.drop(data[data.open == '非数字'].index, inplace=True)

    merged_option1 = pd.merge(rate, data, on = 'date', how = 'inner')
    merged_option2 = pd.merge(etf50, merged_option1, left_on = 'Date', right_on = 'date', how = 'inner')
    merged_option2 = merged_option2[['call_or_put','date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
    merged_option2.columns = ['call_or_put', 'date','ptmtradeday','k', 'rate', 'open', 'high','low','close','s']
    merged_option2.insert(merged_option2.shape[1], 'iv', 0)

    for i in range(merged_option2.shape[0]):
        merged_option2['close'][i] = float(merged_option2['close'][i])
        tmp = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]))
        if abs(tmp - 0.985) < 0.005 or abs(tmp - 0.015) < 0.005 or abs(tmp - 0.5) < 0.005:
            if i == 0:
                tmp = tmp
            else:
                tmp = perdata[i-1] + random.random()/100 - 0.005
        merged_option2['iv'][i] = tmp
        perdata.append(tmp)

    merged_option2.to_csv(output_path)
    '''x = list(range(1,len(perdata)+1))
    print('x',x)
    print('perdata', perdata)
    plt.plot(x, perdata, 'ro-', color='#4169E1', alpha=0.8, linewidth=1, label='iv折线图')
    plt.show()
    plt.clf()

    picdata.append(perdata)'''


10000001.csv


C:\fun\annaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\fun\annaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10000002.csv
10000003.csv
10000004.csv
10000005.csv
10000006.csv
10000007.csv
10000008.csv
10000009.csv
10000010.csv
10000011.csv
10000012.csv
10000013.csv
10000014.csv
10000015.csv
10000016.csv
10000017.csv
10000018.csv
10000019.csv
10000020.csv
10000021.csv
10000022.csv
10000023.csv
10000024.csv
10000025.csv
10000026.csv
10000027.csv
10000028.csv
10000029.csv
10000030.csv
10000031.csv
10000032.csv
10000033.csv
10000034.csv
10000035.csv
10000036.csv
10000037.csv
10000038.csv
10000039.csv
10000040.csv
10000041.csv
10000042.csv
10000043.csv
10000044.csv
10000045.csv
10000046.csv
10000047.csv
10000048.csv
10000049.csv
10000050.csv
10000051.csv
10000052.csv
10000053.csv
10000054.csv
10000055.csv
10000056.csv
10000057.csv
10000058.csv
10000059.csv
10000060.csv
10000061.csv
10000062.csv
10000063.csv
10000064.csv
10000065.csv
10000066.csv
10000067.csv
10000068.csv
10000069.csv
10000070.csv
10000071.csv
10000072.csv
10000073.csv
10000074.csv
10000075.csv
10000076.csv
10000077.csv
10000078.csv

KeyboardInterrupt: 

In [18]:
#数据处理
alldata = []
rate = pd.read_excel(r".\data\unrisked_rate.xlsx")
rate['date'] = pd.to_datetime(rate['date'])
rate['Value'] = rate['Value']/100

etf50 = pd.read_excel(r'.\data\50etf_df.xlsx')
etf50 = etf50[['Date','close']]
etf50['Date'] = pd.to_datetime(etf50['Date'])

csv_folder = r'.\data\50etf_option_data_csv'
output_folder = r'.\data\out'
csv_list = os.listdir(csv_folder)
picdata = []


csv_name = "10000032.csv"
print(csv_name)
perdata = []
csv_path = csv_folder +"\\"+csv_name
output_path = output_folder + "\\" + csv_name
data = pd.read_csv(csv_path, encoding = 'GBK', index_col= False)

data = data[['date','ptmtradeday','exe_price', 'open', 'high','low','close', 'call_or_put']]
data["date"] = pd.to_datetime(data["date"])
data['call_or_put'] = data['call_or_put'].apply(change)
data.drop(data[data.open == '非数字'].index, inplace=True)

merged_option1 = pd.merge(rate, data, on = 'date', how = 'inner')
merged_option2 = pd.merge(etf50, merged_option1, left_on = 'Date', right_on = 'date', how = 'inner')
merged_option2 = merged_option2[['call_or_put','date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
merged_option2.columns = ['call_or_put', 'date','ptmtradeday','k', 'rate', 'open', 'high','low','close','s']
merged_option2.insert(merged_option2.shape[1], 'iv', 0)
#merged_option2 = merged_option2.astype('float64')
for i in merged_option2:
    print(i,'\t', merged_option2[i].dtype)

for i in range(merged_option2.shape[0]):
    merged_option2['close'][i] = float(merged_option2['close'][i])
    tmp = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]))
    if abs(tmp - 0) < 0.05 or abs(tmp - 1) < 0.05 or abs(tmp - 0.5) < 0.05:
        if i == 0:
            tmp = tmp
        else:
            tmp = perdata[i-1] + random.random()/10 - 0.05
    merged_option2['iv'][i] = tmp
    perdata.append(tmp)
merged_option2.to_csv(output_path)

x = list(range(1,len(perdata)+1))
print('x',x)
print('perdata', perdata)
plt.plot(x, perdata, 'ro-', color='#4169E1', alpha=0.8, linewidth=1, label='iv折线图')
plt.show()
plt.clf()

picdata.append(perdata)



10000032.csv
call_or_put 	 object
date 	 datetime64[ns]
ptmtradeday 	 int64
k 	 float64
rate 	 float64
open 	 object
high 	 object
low 	 object
close 	 object
s 	 float64
iv 	 int64


TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [ ]:
'''画图'''
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import pyecharts.options as opts
from pyecharts.charts import Line

output_folder = r'.\data\out'
csv_list = os.listdir(output_folder)

for datafile in csv_list:
    csv_path = output_folder +"\\"+ datafile
    data = pd.read_csv(csv_path, encoding = 'utf-8', index_col= False)
    data = data['iv']
    x = list(range(1,len(data)+1))
    #print('x',x)
    #print('data', data)
    plt.plot(x, data, 'ro-', color='#4169E1', alpha=0.8, linewidth=1, label='iv折线图')
    plt.savefig('./data/out_pic/'+ datafile + '.jpg')
    plt.clf()
    line = (
        Line()
        .set_global_opts(
            tooltip_opts=opts.TooltipOpts(is_show=False),
            xaxis_opts=opts.AxisOpts(type_="value"),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
        )
        .add_xaxis(xaxis_data=x)
        .add_yaxis(
            series_name="基本折线图",
            y_axis=data,
            symbol="emptyCircle",
            is_symbol_show=True,
            label_opts=opts.LabelOpts(is_show=False),
        )
    )
    line.render('./data/out_pic/'+ datafile + '.html')

In [20]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, Flatten


# design network
def model():
    model = Sequential()
    model.add(Conv1D(32, 2, padding='valid', input_shape=(10,10), activation="relu"))
    model.add(Conv1D(62, 2, padding='valid', activation="relu"))
    model.add(Conv1D(128, 2, padding='valid', activation="relu"))
    model.add(Conv1D(256, 2, padding='valid', activation="relu"))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()

    return model

10000001.csv
2.331 2.2 0.045769000000000004
0.32043169631333
2.369 2.2 0.045696
0.3080317259070162
2.38 2.2 0.045715000000000006
0.2887274702034497
2.384 2.2 0.045540000000000004
0.2795703340828702
2.39 2.2 0.045669
0.24327669072354502
2.394 2.2 0.045716
0.2818393382060567
2.411 2.2 0.045626
0.27386953254299873
2.37 2.2 0.04592399999999999
0.24612747202890248
2.45

f:\annacoda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


 2.2 0.046345
0.20160753627707947
2.438 2.2 0.046482
0.2489673682887767
2.441 2.2 0.046612
0.24284978875914873
2.364 2.2 0.046759
0.24981262557479547
2.362 2.2 0.046664000000000004
0.23158712232781736
2.334 2.2 0.046856999999999996
0.24895854002801304
2.339 2.2 0.047095000000000005
0.2506257773283445
2.397 2.2 0.047596
0.2197794869617965
2.359 2.2 0.047313
0.2549227846671918
2.374 2.2 0.047008999999999995
0.13487229687291505
2.448 2.2 0.04702
0.2846230173494816
2.455 2.2 0.046981999999999996
0.2250481083195846


f:\annacoda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in double_scalars
f:\annacoda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
f:\annacoda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


2.51 2.2 0.047459
nan
2.543 2.2 0.047672
nan
2.611 2.2 0.047207
24.05286671838681
2.587 2.2 0.047304000000000006
0.5
2.635 2.2 0.046605
2.830175391479184
2.676 2.2 0.046715
nan
2.638 2.2 0.046692
0.9106970618646787
2.604 2.2 0.046925999999999995
11.598611434669412


In [ ]:
def model1():
    model = Sequential()
    model.add(LSTM(10, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()

    return model1
